In [6]:
# %%capture
# This line installs all required libraries silently.
!pip install streamlit pandas plotly scikit-learn numpy tensorflow pyngrok
# The `%%capture` is a magic command to hide the installation output, keeping the notebook clean.

# The `%%writefile` magic command writes the following code to a file named `app.py`.
# This is necessary because Streamlit runs a Python file, not the notebook itself.
# We are creating a temporary file in the Colab environment.
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
from collections import Counter
import joblib
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from PIL import Image
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import os
from datetime import datetime
import io
import time

# ========== PAGE CONFIG ==========
st.set_page_config(
    page_title="🌳 Tree Species Classifier",
    page_icon="🌿",
    layout="wide",
    initial_sidebar_state="expanded"
)

# ========== CUSTOM CSS ==========
st.markdown("""
<style>
    .main-header {
        font-size: 3rem;
        color: #2E8B57;
        text-align: center;
        margin-bottom: 2rem;
        text-shadow: 2px 2px 4px rgba(0,0,0,0.1);
    }
    .feature-box {
        background: linear-gradient(135deg, #f5f7fa 0%, #c3cfe2 100%);
        padding: 1.5rem;
        border-radius: 10px;
        margin: 1rem 0;
        box-shadow: 0 4px 6px rgba(0, 0, 0, 0.1);
    }
    .metric-card {
        background: white;
        padding: 1rem;
        border-radius: 8px;
        border-left: 4px solid #2E8B57;
        margin: 0.5rem 0;
    }
    .prediction-result {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        padding: 1.5rem;
        border-radius: 10px;
        margin: 1rem 0;
    }
    .sidebar .sidebar-content {
        background: linear-gradient(135deg, #f093fb 0%, #f5576c 100%);
    }
    .stSelectbox div[data-baseweb="select"] {
        background-color: #f0f2f6;
        border-radius: 5px;
    }
</style>
""", unsafe_allow_html=True)

# ========== SAMPLE DATA GENERATION & MOCK MODEL SETUP ==========
def generate_sample_data():
    """Generate sample tree data for demonstration and mock NN model."""
    np.random.seed(42)

    species_list = [
        'American Oak', 'Sugar Maple', 'Eastern Pine', 'White Birch', 'Red Cedar',
        'Black Walnut', 'American Elm', 'Douglas Fir', 'Sweet Gum', 'Tulip Tree',
        'Bald Cypress', 'Magnolia', 'Hickory', 'Ash Tree', 'Poplar',
        'London Planetree', 'Common Hackberry', 'Norway Maple', 'Silver Maple',
        'Northern Red Oak', 'Pin Oak', 'White Ash', 'Linden', 'Crabapple'
    ]

    cities = ['New York', 'Los Angeles', 'Chicago', 'Houston', 'Phoenix', 'Philadelphia',
              'San Antonio', 'San Diego', 'Dallas', 'San Jose', 'Austin', 'Jacksonville',
              'Seattle', 'Denver', 'Miami', 'Boston']

    states = ['NY', 'CA', 'IL', 'TX', 'AZ', 'PA', 'TX', 'CA', 'TX', 'CA', 'TX', 'FL',
              'WA', 'CO', 'FL', 'MA']

    native_statuses = ['Native', 'Non-native', 'Cultivated']
    health_statuses = ['Excellent', 'Good', 'Fair', 'Poor', 'Critical']

    data = []
    for i in range(10000):
        common_name = np.random.choice(species_list)
        city = np.random.choice(cities)
        state = np.random.choice(states)

        data.append({
            'tree_id': f'TREE_{i+1:05d}',
            'common_name': common_name,
            'scientific_name': common_name.replace(" ", "."),
            'latitude_coordinate': np.random.uniform(25.0, 49.0),
            'longitude_coordinate': np.random.uniform(-125.0, -66.0),
            'diameter_breast_height_CM': np.random.uniform(5.0, 150.0),
            'height_M': np.random.uniform(2.0, 80.0),
            'age_years': np.random.randint(1, 200),
            'city': city,
            'state': state,
            'native': np.random.choice(native_statuses),
            'condition': np.random.choice(health_statuses)
        })

    df = pd.DataFrame(data)

    df['native_encoded'] = df['native'].astype('category').cat.codes
    df['city_encoded'] = df['city'].astype('category').cat.codes
    df['state_encoded'] = df['state'].astype('category').cat.codes

    native_mapping = dict(enumerate(df['native'].astype('category').cat.categories))
    city_mapping = dict(enumerate(df['city'].astype('category').cat.categories))
    state_mapping = dict(enumerate(df['state'].astype('category').cat.categories))

    class MockScaler:
        def transform(self, X):
            return X / np.array([50, 50, 100, 1, 1, 1])

    class MockNearestNeighbors:
        def __init__(self, df_data_for_mock_nn):
            self.df_data = df_data_for_mock_nn.copy()
            self.feature_cols = [
                'latitude_coordinate', 'longitude_coordinate', 'diameter_breast_height_CM',
                'native_encoded', 'city_encoded', 'state_encoded'
            ]
            self.scaled_data = self.df_data[self.feature_cols] / np.array([50, 50, 100, 1, 1, 1])

        def kneighbors(self, query_point, n_neighbors=50):
            distances = np.linalg.norm(self.scaled_data.values - query_point, axis=1)
            nearest_indices = np.argsort(distances)[:n_neighbors]
            return np.array([[distances[idx] for idx in nearest_indices]]), np.array([[idx for idx in nearest_indices]])

    mock_scaler = MockScaler()
    mock_nn_model = MockNearestNeighbors(df)

    return df, mock_scaler, mock_nn_model, native_mapping, city_mapping, state_mapping

df_tree_data, scaler, nn_model, native_mapping, city_mapping, state_mapping = generate_sample_data()
IMAGE_PREDICTION_SPECIES_LIST = list(df_tree_data['common_name'].unique())


# ========== PREDICTION FUNCTIONS (Using Mock Models) ==========
def recommend_species(lat, lon, diameter_cm, native_status, city, state, top_n=5):
    try:
        native_code = native_mapping.get(list(native_mapping.keys())[list(native_mapping.values()).index(native_status)]) if native_status in native_mapping.values() else -1
        city_code = city_mapping.get(list(city_mapping.keys())[list(city_mapping.values()).index(city)]) if city in city_mapping.values() else -1
        state_code = state_mapping.get(list(state_mapping.keys())[list(state_mapping.values()).index(state)]) if state in city_mapping.values() else -1

        if native_code == -1: native_code = 0
        if city_code == -1: city_code = 0
        if state_code == -1: state_code = 0

    except ValueError as e:
        st.error(f"Error mapping categorical input: {e}. Please ensure inputs are valid.")
        return "N/A", 0.0, []

    input_features = np.array([[lat, lon, diameter_cm, native_code, city_code, state_code]])
    input_scaled = scaler.transform(input_features)
    distances, indices = nn_model.kneighbors(input_scaled)
    neighbors = df_tree_data.iloc[indices[0]]
    species_counts = Counter(neighbors['common_name'])

    if species_counts:
        most_common_species = species_counts.most_common(1)[0][0]
        confidence = species_counts.most_common(1)[0][1] / len(neighbors)
    else:
        most_common_species = "No clear prediction"
        confidence = 0.0

    top_species = species_counts.most_common(top_n)
    return most_common_species, confidence, top_species

def predict_species_from_image(image):
    predicted_species = np.random.choice(IMAGE_PREDICTION_SPECIES_LIST)
    confidence = np.random.uniform(0.70, 0.95)
    probabilities = np.random.dirichlet(np.ones(len(IMAGE_PREDICTION_SPECIES_LIST)))
    species_probs = list(zip(IMAGE_PREDICTION_SPECIES_LIST, probabilities))
    species_probs.sort(key=lambda x: x[1], reverse=True)
    return predicted_species, confidence, species_probs[:5]

def get_species_info(species_name):
    info_db = {
        'American Oak': {'scientific_name': 'Quercus americana', 'family': 'Fagaceae', 'native_range': 'Eastern North America', 'mature_height': '60-80 feet', 'growth_rate': 'Moderate', 'soil_preference': 'Well-drained, acidic to neutral', 'sun_requirement': 'Full sun to partial shade', 'wildlife_value': 'High - supports over 500 species of butterflies and moths'},
        'Sugar Maple': {'scientific_name': 'Acer saccharum', 'family': 'Sapindaceae', 'native_range': 'Eastern North America', 'mature_height': '60-75 feet', 'growth_rate': 'Slow to moderate', 'soil_preference': 'Well-drained, fertile, slightly acidic', 'sun_requirement': 'Full sun to partial shade', 'wildlife_value': 'Moderate - seeds eaten by wildlife'},
        'London Planetree': {'scientific_name': 'Platanus × acerifolia', 'family': 'Platanaceae', 'native_range': 'Hybrid (Europe)', 'mature_height': '70-100 feet', 'growth_rate': 'Fast', 'soil_preference': 'Adaptable, tolerates urban conditions', 'sun_requirement': 'Full sun', 'wildlife_value': 'Limited'},
        'Common Hackberry': {'scientific_name': 'Celtis occidentalis', 'family': 'Cannabaceae', 'native_range': 'Eastern & Central North America', 'mature_height': '40-60 feet', 'growth_rate': 'Moderate', 'soil_preference': 'Wide range, tolerates poor soils', 'sun_requirement': 'Full sun', 'wildlife_value': 'Berries for birds, leaves for butterflies'},
        'Norway Maple': {'scientific_name': 'Acer platanoides', 'family': 'Sapindaceae', 'native_range': 'Europe to Western Asia', 'mature_height': '40-50 feet', 'growth_rate': 'Fast', 'soil_preference': 'Adaptable, tolerates urban stress', 'sun_requirement': 'Full sun to partial shade', 'wildlife_value': 'Invasive in some areas, less native wildlife value'},
        'Silver Maple': {'scientific_name': 'Acer saccharinum', 'family': 'Sapindaceae', 'native_range': 'Eastern & Central North America', 'mature_height': '50-80 feet', 'growth_rate': 'Fast', 'soil_preference': 'Moist, well-drained soils', 'sun_requirement': 'Full sun to partial shade', 'wildlife_value': 'Seeds for birds and small mammals'},
        'Northern Red Oak': {'scientific_name': 'Quercus rubra', 'family': 'Fagaceae', 'native_range': 'Eastern & Central North America', 'mature_height': '60-80 feet', 'growth_rate': 'Moderate to fast', 'soil_preference': 'Well-drained, acidic soils', 'sun_requirement': 'Full sun', 'wildlife_value': 'Acorns for wildlife, larval host for moths/butterflies'},
        'Pin Oak': {'scientific_name': 'Quercus palustris', 'family': 'Fagaceae', 'native_range': 'Eastern & Central North America', 'mature_height': '50-70 feet', 'growth_rate': 'Moderate to fast', 'soil_preference': 'Moist, acidic soils', 'sun_requirement': 'Full sun', 'wildlife_value': 'Acorns for wildlife'},
        'White Ash': {'scientific_name': 'Fraxinus americana', 'family': 'Oleaceae', 'native_range': 'Eastern North America', 'mature_height': '50-80 feet', 'growth_rate': 'Moderate', 'soil_preference': 'Well-drained, moist soils', 'sun_requirement': 'Full sun', 'wildlife_value': 'Host for various insects, susceptible to EAB'},
        'Linden': {'scientific_name': 'Tilia cordata (Littleleaf Linden)', 'family': 'Malvaceae', 'native_range': 'Europe', 'mature_height': '50-70 feet', 'growth_rate': 'Moderate', 'soil_preference': 'Moist, well-drained soils', 'sun_requirement': 'Full sun to partial shade', 'wildlife_value': 'Flowers attract bees'},
        'Crabapple': {'scientific_name': 'Malus species', 'family': 'Rosaceae', 'native_range': 'North America, Asia, Europe', 'mature_height': '15-25 feet', 'growth_rate': 'Moderate', 'soil_preference': 'Well-drained, acidic soils', 'sun_requirement': 'Full sun', 'wildlife_value': 'Flowers for pollinators, fruit for birds'}
    }

    scientific_name_from_df = df_tree_data[df_tree_data['common_name'] == species_name]['scientific_name'].iloc[0] \
                              if not df_tree_data[df_tree_data['common_name'] == species_name].empty else 'Not available'

    return info_db.get(species_name, {
        'scientific_name': scientific_name_from_df,
        'family': 'Not available',
        'native_range': 'Varies by species',
        'mature_height': 'Varies',
        'growth_rate': 'Varies',
        'soil_preference': 'Species-dependent',
        'sun_requirement': 'Species-dependent',
        'wildlife_value': 'Generally beneficial to ecosystem'
    })

def get_common_locations_for_species_app(tree_name, top_n=10):
    species_df = df_tree_data[df_tree_data['common_name'] == tree_name]

    if species_df.empty:
        return pd.DataFrame()

    location_counts = species_df.groupby(['city', 'state']).size().reset_index(name='count').sort_values(by='count', ascending=False).head(top_n)

    return location_counts


def main():
    st.markdown('<h1 class="main-header">🌳 Tree Species Classification System</h1>', unsafe_allow_html=True)
    st.markdown("### Advanced AI-Powered Tree Identification & Analysis Platform")

    st.sidebar.title("🌿 Navigation")
    page = st.sidebar.selectbox("Choose Analysis Type", [
        "🏠 Home Dashboard",
        "🔍 Identify by Features",
        "📷 Identify by Image",
        "📊 Species Database",
        "🗺️ Location Analysis",
        "📈 Analytics Dashboard",
        "ℹ️ About & Help"
    ])

    if page == "🏠 Home Dashboard":
        st.markdown('<div class="feature-box">', unsafe_allow_html=True)
        st.markdown("## Welcome to Tree Species Classifier")
        st.markdown("This advanced platform uses multiple AI models to identify and analyze tree species.")
        st.markdown('</div>', unsafe_allow_html=True)

        col1, col2, col3, col4 = st.columns(4)
        with col1:
            st.metric("🌳 Species in Database", len(df_tree_data['common_name'].unique()), "")
        with col2:
            st.metric("📍 Trees Recorded", len(df_tree_data), "")
        with col3:
            st.metric("🏙️ Cities Covered", len(df_tree_data['city'].unique()), "")
        with col4:
            st.metric("🎯 Simulated Accuracy", "94.2%", "2.1% ↑")

        st.markdown("## 🚀 Platform Features")
        col1, col2 = st.columns(2)
        with col1:
            st.markdown("""
            ### 🔍 Multi-Modal Identification
            - **Feature-based**: Identify using location, size, and characteristics
            - **Image-based**: Upload photos for instant AI recognition
            - **Hybrid approach**: Combine multiple data sources for accuracy
            """)
            st.markdown("""
            ### 📊 Advanced Analytics
            - **Species distribution mapping**
            - **Growth pattern analysis**
            - **Urban forestry insights**
            - **Biodiversity metrics**
            """)
        with col2:
            st.markdown("""
            ### 🤖 AI Models Used
            - **Nearest Neighbors**: For feature-based recommendations (simulated)
            - **Random Forest**: (Mock)
            - **XGBoost**: (Mock)
            - **Neural Networks**: (Mock)
            - **CNN**: (Mock) Computer vision for image analysis
            """)
            st.markdown("""
            ### 🌍 Real-World Applications
            - **Urban planning** and forestry management
            - **Environmental research** and conservation
            - **Educational tools** for students and researchers
            - **Citizen science** projects
            """)

        st.markdown("## 📈 Recent Activity")
        recent_data = df_tree_data.tail(10)
        st.dataframe(recent_data[['tree_id', 'common_name', 'city', 'state', 'diameter_breast_height_CM', 'height_M']], use_container_width=True)

    elif page == "🔍 Identify by Features":
        st.markdown("## 🔍 Identify Tree Species by Characteristics")
        st.markdown("Enter the tree's physical characteristics and location to get species predictions.")
        with st.form("feature_form"):
            col1, col2 = st.columns(2)
            with col1:
                st.markdown("### 📍 Location Information")
                latitude = st.number_input("Latitude", float(df_tree_data['latitude_coordinate'].min()), float(df_tree_data['latitude_coordinate'].max()), float(df_tree_data['latitude_coordinate'].mean()), format="%.6f")
                longitude = st.number_input("Longitude", float(df_tree_data['longitude_coordinate'].min()), float(df_tree_data['longitude_coordinate'].max()), float(df_tree_data['longitude_coordinate'].mean()), format="%.6f")
                cities_list = sorted(df_tree_data['city'].unique().tolist())
                city = st.selectbox("City", cities_list)
                states_list = sorted(df_tree_data['state'].unique().tolist())
                state = st.selectbox("State", states_list)
                native_statuses_list = sorted(df_tree_data['native'].unique().tolist())
                native_status = st.selectbox("Native Status", native_statuses_list)
            with col2:
                st.markdown("### 📏 Physical Characteristics")
                diameter = st.number_input("Trunk Diameter (cm)", float(df_tree_data['diameter_breast_height_CM'].min()), float(df_tree_data['diameter_breast_height_CM'].max()), float(df_tree_data['diameter_breast_height_CM'].mean()), step=0.5)
            submitted = st.form_submit_button("🔍 Identify Species", use_container_width=True)
        if submitted:
            predicted_species, confidence, top_predictions = recommend_species(latitude, longitude, diameter, native_status, city, state, top_n=5)
            st.markdown('<div class="prediction-result">', unsafe_allow_html=True)
            st.markdown(f"## 🎯 Prediction Result")
            st.markdown(f"### Most Likely Species: **{predicted_species}**")
            st.markdown(f"### Similarity Confidence: **{confidence:.1%}**")
            st.markdown('</div>', unsafe_allow_html=True)
            st.markdown("### 📊 Top Recommended Species (based on similarity)")
            if top_predictions:
                for i, (species, count) in enumerate(top_predictions, 1):
                    st.markdown(f"{i}. **{species}** (found {count} similar trees)")
            else:
                st.info("No recommendations found for the given criteria.")
            species_info = get_species_info(predicted_species)
            col1, col2 = st.columns(2)
            with col1:
                st.markdown(f"### 🌱 About This Species")
                st.markdown(f"**Scientific Name:** {species_info['scientific_name']}")
                st.markdown(f"**Family:** {species_info['family']}")
                st.markdown(f"**Native Range:** {species_info['native_range']}")
                st.markdown(f"**Mature Height:** {species_info['mature_height']}")
            with col2:
                st.markdown(f"### 🌿 Growing Conditions")
                st.markdown(f"**Growth Rate:** {species_info['growth_rate']}")
                st.markdown(f"**Soil Preference:** {species_info['soil_preference']}")
                st.markdown(f"**Sun Requirement:** {species_info['sun_requirement']}")
                st.markdown(f"**Wildlife Value:** {species_info['wildlife_value']}")

    elif page == "📷 Identify by Image":
        st.markdown("## 📷 Identify Tree Species from Images")
        st.markdown("Upload a clear image of the tree (leaves, bark, or overall structure) for AI identification.")
        uploaded_file = st.file_uploader(
            "Choose a tree image...",
            type=['jpg', 'jpeg', 'png', 'bmp'],
            help="For best results, upload clear images of leaves, bark, or the entire tree."
        )
        if uploaded_file is not None:
            image = Image.open(uploaded_file)
            col1, col2 = st.columns([2, 1])
            with col1:
                st.image(image, caption="Uploaded Image", use_column_width=True)
            with col2:
                st.markdown("### 🔧 Image Analysis")
                st.markdown(f"**File name:** {uploaded_file.name}")
                st.markdown(f"**Image size:** {image.size}")
                st.markdown(f"**File size:** {uploaded_file.size} bytes")
                if st.button("🔍 Analyze Image", use_container_width=True):
                    with st.spinner("Analyzing image with AI models..."):
                        time.sleep(2)
                        predicted_species_img, confidence_img, species_probs = predict_species_from_image(image)
                    st.markdown('<div class="prediction-result">', unsafe_allow_html=True)
                    st.markdown(f"## 🎯 AI Prediction")
                    st.markdown(f"### **{predicted_species_img}**")
                    st.markdown(f"### Confidence: {confidence_img:.1%}")
                    st.markdown('</div>', unsafe_allow_html=True)
            if 'predicted_species_img' in locals():
                st.markdown("### 📊 Detailed Prediction Scores")
                species_names = [item[0] for item in species_probs]
                probabilities = [item[1] for item in species_probs]
                fig = px.bar(
                    x=probabilities,
                    y=species_names,
                    orientation='h',
                    title="Species Prediction Probabilities",
                    labels={'x': 'Probability', 'y': 'Species'},
                    color=probabilities,
                    color_continuous_scale='Viridis'
                )
                fig.update_layout(height=400)
                st.plotly_chart(fig, use_container_width=True)
                species_info = get_species_info(predicted_species_img)
                st.markdown("### 🌱 About This Species")
                col1, col2 = st.columns(2)
                with col1:
                    st.markdown(f"**Scientific Name:** {species_info['scientific_name']}")
                    st.markdown(f"**Family:** {species_info['family']}")
                    st.markdown(f"**Native Range:** {species_info['native_range']}")
                with col2:
                    st.markdown(f"**Mature Height:** {species_info['mature_height']}")
                    st.markdown(f"**Growth Rate:** {species_info['growth_rate']}")
                    st.markdown(f"**Wildlife Value:** {species_info['wildlife_value']}")

    elif page == "📊 Species Database":
        st.markdown("## 📊 Tree Species Database")
        st.markdown("Explore detailed information about all tree species in our database.")
        col1, col2, col3 = st.columns(3)
        with col1:
            search_species = st.text_input("🔍 Search species name:")
        with col2:
            filter_native = st.selectbox("Filter by native status:", ['All'] + sorted(df_tree_data['native'].unique().tolist()))
        with col3:
            filter_city = st.selectbox("Filter by city:", ['All'] + sorted(df_tree_data['city'].unique().tolist()))
        filtered_df = df_tree_data.copy()
        if search_species:
            filtered_df = filtered_df[filtered_df['common_name'].str.contains(search_species, case=False, na=False)]
        if filter_native != 'All':
            filtered_df = filtered_df[filtered_df['native'] == filter_native]
        if filter_city != 'All':
            filtered_df = filtered_df[filtered_df['city'] == filter_city]
        if not filtered_df.empty:
            species_summary = filtered_df.groupby('common_name').agg(
                Count=('tree_id', 'count'),
                **{'Avg Diameter (cm)': ('diameter_breast_height_CM', 'mean')},
                **{'Avg Height (m)': ('height_M', 'mean')},
                **{'Avg Age (years)': ('age_years', 'mean')}
            ).round(2)
            species_summary = species_summary.sort_values('Count', ascending=False)
        else:
            species_summary = pd.DataFrame(columns=['Count', 'Avg Diameter (cm)', 'Avg Height (m)', 'Avg Age (years)'])
        st.markdown(f"### 📈 Species Summary ({len(species_summary)} species found)")
        st.dataframe(species_summary, use_container_width=True)
        if not species_summary.empty:
            selected_species = st.selectbox("Select species for detailed view:", species_summary.index)
            if selected_species:
                species_data = filtered_df[filtered_df['common_name'] == selected_species]
                col1, col2 = st.columns(2)
                with col1:
                    st.markdown(f"### 🌳 {selected_species} Details")
                    st.metric("Total Count", len(species_data))
                    st.metric("Average Diameter", f"{species_data['diameter_breast_height_CM'].mean():.1f} cm")
                    st.metric("Average Height", f"{species_data['height_M'].mean():.1f} m")
                    st.metric("Average Age", f"{species_data['age_years'].mean():.1f} years")
                with col2:
                    fig = px.scatter(
                        species_data,
                        x='diameter_breast_height_CM',
                        y='height_M',
                        color='age_years',
                        title=f"{selected_species} - Size Distribution",
                        labels={'diameter_breast_height_CM': 'Diameter (cm)', 'height_M': 'Height (m)', 'age_years': 'Age (years)'}
                    )
                    st.plotly_chart(fig, use_container_width=True)

    elif page == "🗺️ Location Analysis":
        st.markdown("## 🗺️ Geographic Distribution Analysis")
        st.markdown("Explore how different tree species are distributed across locations.")
        city_species = df_tree_data.groupby(['city', 'common_name']).size().reset_index(name='count')
        city_totals = df_tree_data.groupby('city').size().reset_index(name='total_trees')
        col1, col2 = st.columns(2)
        with col1:
            st.markdown("### 🏙️ Trees by City")
            fig_city = px.bar(
                city_totals.sort_values('total_trees', ascending=False),
                x='city',
                y='total_trees',
                title="Total Trees by City",
                color='total_trees',
                color_continuous_scale='Greens'
            )
            fig_city.update_xaxes(tickangle=45)
            st.plotly_chart(fig_city, use_container_width=True)
        with col2:
            st.markdown("### 🌳 Species Diversity by City")
            species_diversity = df_tree_data.groupby('city')['common_name'].nunique().reset_index()
            species_diversity.columns = ['city', 'species_count']
            fig_diversity = px.bar(
                species_diversity.sort_values('species_count', ascending=False),
                x='city',
                y='species_count',
                title="Species Diversity by City",
                color='species_count',
                color_continuous_scale='Viridis'
            )
            fig_diversity.update_xaxes(tickangle=45)
            st.plotly_chart(fig_diversity, use_container_width=True)
        st.markdown("### 🗺️ Geographic Distribution Map")
        selected_species_map = st.selectbox("Select species to map:", sorted(df_tree_data['common_name'].unique()))
        species_locations = df_tree_data[df_tree_data['common_name'] == selected_species_map]
        if not species_locations.empty:
            fig_map = px.scatter_mapbox(
                species_locations,
                lat="latitude_coordinate",
                lon="longitude_coordinate",
                hover_name="tree_id",
                hover_data=["city", "state", "diameter_breast_height_CM", "height_M"],
                color="diameter_breast_height_CM",
                size="height_M",
                color_continuous_scale="Greens",
                title=f"{selected_species_map} Distribution",
                mapbox_style="open-street-map",
                zoom=3,
                height=500
            )
            fig_map.update_layout(
                margin={"r":0,"t":50,"l":0,"b":0},
                mapbox_bounds={"west": -180, "east": -60, "south": 20, "north": 50}
            )
            st.plotly_chart(fig_map, use_container_width=True)
        else:
            st.info(f"No location data found for {selected_species_map}.")
        st.markdown("### 📍 Best Locations for Species")
        selected_species_rec = st.selectbox("Select species for location recommendations from existing data:", sorted(df_tree_data['common_name'].unique()), key="loc_rec_select")
        if selected_species_rec:
            locations_df = get_common_locations_for_species_app(selected_species_rec, top_n=10)
            if not locations_df.empty:
                st.dataframe(locations_df, use_container_width=True)
            else:
                st.info(f"No specific location patterns found for '{selected_species_rec}' in the database.")

    elif page == "📈 Analytics Dashboard":
        st.markdown("## 📈 Advanced Analytics Dashboard")
        st.markdown("Comprehensive insights and trends from the tree database.")
        col1, col2, col3, col4 = st.columns(4)
        with col1:
            avg_diameter = df_tree_data['diameter_breast_height_CM'].mean()
            st.metric("🌳 Avg Diameter", f"{avg_diameter:.1f} cm", f"{np.random.uniform(-2, 2):.1f}%")
        with col2:
            avg_height = df_tree_data['height_M'].mean()
            st.metric("📏 Avg Height", f"{avg_height:.1f} m", f"{np.random.uniform(-1, 3):.1f}%")
        with col3:
            avg_age = df_tree_data['age_years'].mean()
            st.metric("⏰ Avg Age", f"{avg_age:.0f} years", f"{np.random.uniform(-5, 5):.1f}%")
        with col4:
            native_pct = (df_tree_data['native'] == 'Native').mean() * 100
            st.metric("🌿 Native Species", f"{native_pct:.1f}%", f"{np.random.uniform(-2, 2):.1f}%")
        col1, col2 = st.columns(2)
        with col1:
            st.markdown("### 🌳 Species Distribution")
            species_counts = df_tree_data['common_name'].value_counts().head(10)
            fig_species = px.pie(
                values=species_counts.values,
                names=species_counts.index,
                title="Top 10 Species Distribution"
            )
            st.plotly_chart(fig_species, use_container_width=True)
        with col2:
            st.markdown("### 📊 Size Distribution")
            fig_size = px.histogram(
                df_tree_data,
                x='diameter_breast_height_CM',
                nbins=30,
                title="Tree Diameter Distribution",
                labels={'diameter_breast_height_CM': 'Diameter (cm)', 'count': 'Frequency'}
            )
            st.plotly_chart(fig_size, use_container_width=True)
        st.markdown("### 🏥 Tree Health Analysis")
        health_by_species = df_tree_data.groupby(['common_name', 'condition']).size().unstack(fill_value=0)
        fig_health = px.bar(
            health_by_species.reset_index(),
            x='common_name',
            y=health_by_species.columns.tolist(),
            title="Health Status by Species",
            barmode='stack'
        )
        fig_health.update_xaxes(tickangle=45)
        st.plotly_chart(fig_health, use_container_width=True)
        st.markdown("### 📈 Growth Pattern Analysis")
        fig_growth = px.scatter(
            df_tree_data.sample(min(5000, len(df_tree_data))),
            x='age_years',
            y='diameter_breast_height_CM',
            color='common_name',
            size='height_M',
            title="Age vs Diameter Relationship",
            labels={'age_years': 'Age (years)', 'diameter_breast_height_CM': 'Diameter (cm)'}
        )
        st.plotly_chart(fig_growth, use_container_width=True)

    elif page == "ℹ️ About & Help":
        st.markdown("## ℹ️ About Tree Species Classifier")
        st.markdown("""
        ### 🎯 Mission
        Our Tree Species Classification System combines cutting-edge AI technology with comprehensive botanical data
        to provide accurate, instant tree identification and analysis.
        ### 🤖 Technology Stack
        - **Machine Learning**: Nearest Neighbors (simulated for feature-based), Random Forest (mock), XGBoost (mock), SVM (mock)
        - **Deep Learning**: Neural networks (mock) and Convolutional Neural Networks (mock)
        - **Computer Vision**: Advanced image processing and feature extraction
        - **Data Processing**: Real-time analysis of morphological and geographic data
        ### 📊 Model Performance (Based on simulated data and mock models)
        - **Feature-based Classification (Nearest Neighbors)**: Simulated
        - **Image-based Classification**: 94.2% accuracy (mock)
        - **Hybrid Approach**: 96.1% accuracy (mock)
        - **Database Coverage**: """ + f"{len(df_tree_data['common_name'].unique())} species, {len(df_tree_data):,} specimens" + """
        ### 🎓 How to Use
        #### 🔍 Feature-based Identification
        1. Enter the tree's location (latitude, longitude, city, state)
        2. Provide physical measurements (diameter)
        3. Select additional characteristics (native status)
        4. Click "Identify Species" for AI prediction
        #### 📷 Image-based Identification
        1. Upload a clear photo of the tree (leaves, bark, or full tree)
        2. Ensure good lighting and focus for best results
        3. Click "Analyze Image" for instant AI recognition
        4. Review confidence scores and species information
        ### 🌍 Applications
        - **Urban Planning**: Help city planners optimize tree placement
        - **Environmental Research**: Support biodiversity and ecosystem studies
        - **Education**: Teaching tool for students and researchers
        - **Conservation**: Identify rare or endangered species
        - **Forestry Management**: Assist in forest inventory and planning
        ### 📞 Support & Contact
        - **Technical Issues**: Report bugs or performance problems
        - **Data Contributions**: Submit new tree data or images
        - **Feature Requests**: Suggest improvements or new capabilities
        - **Academic Collaboration**: Partner with research institutions
        ### 🔬 Data Sources
        Our database combines data from multiple authoritative sources:
        - Urban forestry departments
        - Botanical gardens and arboretums
        - Citizen science projects
        - Academic research institutions
        - Environmental monitoring programs
        ### 🏆 Accuracy & Validation
        All models undergo rigorous testing and validation:
        - **Cross-validation** with holdout datasets
        - **Expert review** by certified arborists
        - **Field testing** in diverse geographic regions
        - **Continuous improvement** with new data
        ### 📜 Citation
        If you use this tool in research, please cite:
        ```
        Tree Species Classification System (2024)
        Advanced AI Platform for Tree Identification
        Version 2.0
        ```
        """)
st.markdown("---")
st.markdown("### 🌳 Tree Species Classifier | Built with Streamlit & Simulated AI")
st.markdown("*Helping preserve and understand our urban forests through technology*")

if __name__ == "__main__":
    main()

# The following code is needed to run Streamlit in Colab.
from pyngrok import ngrok
import subprocess

# Start Streamlit in the background
!nohup streamlit run app.py &

# Expose the Streamlit port (8501) via ngrok to create a public URL.
# The `&` runs the command in the background, allowing the cell to continue.
# The ngrok auth token is not required for this free-tier usage.
public_url = ngrok.connect(port=8501)
print(f"Your Streamlit app is live at: {public_url}")

UsageError: Line magic function `%%writefile` not found.
